In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler

/Users/John/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df = pd.read_csv('data/train.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14993 entries, 0 to 14992
Data columns (total 24 columns):
Type             14993 non-null int64
Name             13736 non-null object
Age              14993 non-null int64
Breed1           14993 non-null int64
Breed2           14993 non-null int64
Gender           14993 non-null int64
Color1           14993 non-null int64
Color2           14993 non-null int64
Color3           14993 non-null int64
MaturitySize     14993 non-null int64
FurLength        14993 non-null int64
Vaccinated       14993 non-null int64
Dewormed         14993 non-null int64
Sterilized       14993 non-null int64
Health           14993 non-null int64
Quantity         14993 non-null int64
Fee              14993 non-null int64
State            14993 non-null int64
RescuerID        14993 non-null object
VideoAmt         14993 non-null int64
Description      14981 non-null object
PetID            14993 non-null object
PhotoAmt         14993 non-null float64
AdoptionSpe

In [4]:
X = df.drop(columns=['Name','RescuerID','Description','PetID','AdoptionSpeed'])
y = df['AdoptionSpeed'].astype('str')

In [5]:
sclr = StandardScaler()
X = sclr.fit_transform(X)

In [6]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [7]:
log = LogisticRegression(multi_class='ovr')
log.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [8]:
log_pred = log.predict(X_val)
np.sum(log_pred==y_val) / len(y_val)

0.3584528176058686

In [9]:
gb = GradientBoostingClassifier(subsample=0.9)
gb.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=0.9, verbose=0,
              warm_start=False)

In [10]:
gb_pred = gb.predict(X_val)
np.sum(gb_pred==y_val) / len(y_val)

0.40346782260753583

In [11]:
enc = OneHotEncoder()
y_train_enc = enc.fit_transform(y_train.reshape(-1,1))

/Users/John/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


In [43]:
nn = Sequential()
nn.add(Dense(128, activation='relu', input_dim=(X_train.shape[1])))
nn.add(Dropout(0.5))
nn.add(Dense(64, activation='relu'))
nn.add(Dropout(0.5))
nn.add(Dense(32, activation='relu'))
nn.add(Dropout(0.7))
nn.add(Dense(16, activation='relu'))
nn.add(Dropout(0.7))
nn.add(Dense(5,activation='sigmoid'))

In [44]:
nn.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [45]:
nn.fit(X_train, y_train_enc, epochs=20)

Epoch 1/20
11994/11994 [==============================] - 1s 107us/step - loss: 1.6412 - acc: 0.2326
Epoch 2/20
11994/11994 [==============================] - 1s 63us/step - loss: 1.5639 - acc: 0.2552
Epoch 3/20
11994/11994 [==============================] - 1s 61us/step - loss: 1.5406 - acc: 0.2524
Epoch 4/20
11994/11994 [==============================] - 1s 61us/step - loss: 1.5164 - acc: 0.2574
Epoch 5/20
11994/11994 [==============================] - 1s 61us/step - loss: 1.5055 - acc: 0.2627
Epoch 6/20
11994/11994 [==============================] - 1s 63us/step - loss: 1.4986 - acc: 0.2585
Epoch 7/20
11994/11994 [==============================] - 1s 62us/step - loss: 1.4919 - acc: 0.2653
Epoch 8/20
11994/11994 [==============================] - 1s 62us/step - loss: 1.4871 - acc: 0.2709
Epoch 9/20
11994/11994 [==============================] - 1s 61us/step - loss: 1.4829 - acc: 0.2728
Epoch 10/20
11994/11994 [==============================] - 1s 62us/step - loss: 1.4789 - acc: 0.269

In [49]:
nn_pred = nn.predict(X_val)
np.sum(enc.inverse_transform(nn_pred)==y_val) / len(y_val)

AttributeError: 'OneHotEncoder' object has no attribute 'inverse_transform'

In [50]:
enc.inverse_transform(nn_pred)

AttributeError: 'OneHotEncoder' object has no attribute 'inverse_transform'